In [34]:
import pandas as pd
import operator

In [35]:
columns = ["name", "movie", "date", "rating"]
netflix = pd.read_csv("./database/com402-3.csv", skipinitialspace=True, quotechar='"', header=None, names=columns)
imdb = pd.read_csv("./database/imdb-3.csv", skipinitialspace=True, quotechar='"', header=None, names=columns)

netflix["date"] = pd.to_datetime(netflix["date"], dayfirst =True)
imdb["date"] = pd.to_datetime(imdb["date"], dayfirst =True)

netflix.head()

,name,movie,date,rating
0,0be567e3ce47645262aac4b77d6911861cf1dfdfda2fa2...,9531c709a8ffb3ee40218af7ee3bf857d5d867c0436314...,2001-01-01,1
1,496f408ec7b099c2d269ddb122862f965e138b4ba9f7a6...,8e31cb88686e36651c4e0d786efafa819874f710314d88...,2001-01-01,1
2,806ed8f42fd3754d270c79c1b0a5b5920fe1ca3f6380fb...,ae551c1d604e8591e4578dbb598a966b6177c9485b5b62...,2001-01-01,4
3,b22b2115bd41a995b031d6c615673abf26144f82debe48...,9dea21681a8b8e5dbe498adac1c2fbb0af1c149c88fc21...,2001-01-01,3
4,459a1531a3feefaf99498062931d2b27d5b756082d1689...,c3aa702b04623b78d92f30cdc44f6bea247421e09a8cc2...,2001-01-01,4


In [36]:
my_imdb_movies = imdb[imdb["name"] == "martin.vold@epfl.ch"]
my_dates = my_imdb_movies["date"].values

# Finding all hashed rows within 14 days from a rating date in imdb rating
possible_hashes = []
for date in my_dates:
    for i, netflix_row in netflix.iterrows():
        diff = (netflix_row["date"] - date).days
        if abs(diff) < 14:
            possible_hashes.append(netflix_row.tolist())
            
possible_hashes = pd.DataFrame(possible_hashes, columns=columns)
my_hash = possible_hashes.name.value_counts().index[0]

In [37]:
my_hash

'2d252020bd08e25a46a7a722896bf2706aae500f20a3b7d71213ccbd8a3816be'

In [38]:
my_netflix_movies = netflix[netflix["name"] == my_hash]
my_movies = []

# Find all movies that corespond to my movie hashes
for hashed_movie in my_netflix_movies["movie"].values:
    possible_movies_scores = {}
    
    for hashed_date in netflix[netflix["movie"] == hashed_movie].date:
        upper_hashed_date = hashed_date + pd.Timedelta(days=13)
        lower_hashed_date = hashed_date - pd.Timedelta(days=13)
        possible_imdb_movies_mask = (imdb.date <= upper_hashed_date) & (imdb.date >= lower_hashed_date)
        possible_imdb_movies = imdb[possible_imdb_movies_mask]
        
        for i, possible_movie in possible_imdb_movies.iterrows():
            diff = abs((hashed_date - possible_movie["date"]).days)
            if possible_movie.movie in possible_movies_scores.keys():
                possible_movies_scores[possible_movie.movie] += 14 - diff
            else:
                possible_movies_scores[possible_movie.movie] = 14 - diff
    
    # Bet movie with highest count
    best_movie = max(possible_movies_scores, key=possible_movies_scores.get)
    
    my_movies.append(best_movie)

In [39]:
f = open("ex1c.txt", "a")
for movie in my_movies:
    f.write(movie+"\n")
f.close()